## Turbo kolesar

### 1. Premik in lokacija

Napišite razred `Kolesar`. Ta naj ima

- konstruktor (`__init__(self)`), ki naredi, kar je potrebno, da bo razred deloval;
- metodo `premik(self, smer)`, ki prejme niz smer, ki je lahko `"<"`, `">"`, `"^"` ali `"v"` in premakne kolesarja za eno polje levo, desno, gor ali dol in
- metodo `lokacija(self)`, ki vrne trenutne koordinate (x, y) kolesarja.

Kolesar je v začetku na lokaciji (0, 0). Koordinatni sistem je obrnjen tako, da premik gor (`^`) *zmanjša* `y`.

```python
ana = Kolesar()
ana.premik(">")
print(ana.lokacija())
ana.premik("^")
print(ana.lokacija())
```

izpiše

```python
(1, 0)
(1, 1)
```

#### Rešitev

Ugotoviti moramo, kakšne stvari bo potrebno shranjevati in se odločiti bomo poimenovali atribute. Očitno bo potrebno shraniti koordinate in primerno imeni sta `x` in `y`. Potem že vemo, kakšna bosta konstruktor in metoda lokacija.

In [ ]:
class Kolesar:
    def __init__(self):
        self.x = 0
        self.y = 0

    def lokacija(self):
        return (self.x, self.y)

Tudi `premik` ni posebna znanost, je pa zanimivo pogledati nekaj različic. Tule so. :)

In [1]:
def premik(self, smer):
    if smer == "<":
        self.x -= 1
    elif smer == ">":
        self.x += 1
    elif smer == "v":
        self.y += 1
    elif smer == "^":
        self.y -= 1

def premik(self, smer):
    dx, dy = {"<": (-1, 0), ">": (1, 0), "v": (0, 1), "^": (0, -1)}[smer]
    self.x += dx
    self.y += dy

def premik(self, smer):
    x, y = self.x, self.y
    self.x, self.y ={"<": (x - 1, y), ">": (x + 1, y), "v": (x, y + 1), "^": (x, y - 1)}

Prva različica je "dolgočasna", napisal bi jo nekdo, ki je vajen nizkonivojskih jezikov in ob tem bi se pritoževal, zakaj Python nima stavka `switch`. (Ima ga, imenuje se `match`.)

Druga je bolj v slogu višjenivojskih jeziko, v katere so slovarji vgrajeni tesneje in jih zato radi uporabljamo tudi tam, kjer v C++ ali Javi nikoli ne bi pomislili na `map` ali `HashMap` ali kaj podobnega.

Tretja je podobna drugi, le malo potratnejša.

V razredu za hec uporabimo četrto, ki se igra z dejstvom, da je `True` toliko kot `1` in `False` toliko kot `0`.

In [3]:
class Kolesar:
    def __init__(self):
        self.x = 0
        self.y = 0

    def premik(self, smer):
        self.x += (smer == ">") - (smer == "<")
        self.y += (smer == "v") - (smer == "^")

    def lokacija(self):
        return self.x, self.y

V nekaterih jezikih je to prepovedano: `bool` je `bool` in `int` je `int`. `True` tam **ni** `1`.

Ti jeziki imajo prav. Zato to niso samo *nekateri jeziki*, ampak je to zadnje čase kar v modi; takšnih stvari nam ne bo pustil ne Kotlin ne Typescript. Ampak ... ko si v Rimu, se obnašaj kot Rimljani. Ko programiram v C-ju in Pythonu pač počnem stvari, ki jih v Kotlinu in Typescriptu ne bi. :)

### 2. Prevozi

Kolesarju dodajte metodo `prevozi(pot)`, ki prejme pot oblike, na primer, `5<3^2^8>2<2v`. Pot, torej je sestavljena iz zaporedja števk (enomestnih številk) in znakov, ki kažejo smer. Dobivši niz iz primera mora kolesar petkrat levo, trikrat gor, dvakrat gor, osemkrat desno, dvakrat levo in dvakrat gor.

Nasvet: metoda `prevozi` naj kliče metodo `premik`.

```python
ana = Kolesar()
ana.premik(">")
ana.prevozi("7^4<")
ana.premik(">")
print(ana.lokacija())
```

izpiše `(2, -7)`.

#### Rešitev

Tu bi lahko dopolnili premik z dodatnim argumentom, s katerim bi povedali število korakov in mu dali privzeto vrednost 0. To bi bilo časovno učinkovitejše, vendar bi nam zapletlo življenje v naslednjih nalogah, zato bomo raje večkrat poklicali `premik`.

Glavni trik je, kako razdreti niz s potjo. Ker se števke in smeri izmenjujejo, je najpreprosteje kar vzeti vsak drug element - enkrat začenši z ničtim, enkrat s prvim, `pot[::2]`, `pot[1::2]`. To potem zipnemo skupaj, pa dobimo pare s števkami in smermi.

In [6]:
pot = "5<3^2^8>2<2v"

list(zip(pot[::2], pot[1::2]))

[('5', '<'), ('3', '^'), ('2', '^'), ('8', '>'), ('2', '<'), ('2', 'v')]

Tule smo `list` poklicali le zato, ker `zip` vrne iterator; s klicem `list` ga prisilimo, da "iziterira" elemente in se ti lahko izpišejo. Če bomo šli čez pare v zanki, to seveda ni potrebno.

In [7]:
    def prevozi(self, pot):
        for koliko, kam in zip(pot[::2], pot[1::2]):
            for _ in range(int(koliko)):
                self.premik(kam)

Med pisanjem tega besedila sem se spomnil, da so v Pythonu 3.12 dodali funkcijo `itertools.batched`, s katero dosežemo isto.

In [10]:
from itertools import batched

list(batched(pot, 2))

[('5', '<'), ('3', '^'), ('2', '^'), ('8', '>'), ('2', '<'), ('2', 'v')]

### 3. Razdalja

Kolesarju dodajte metodo `razdalja()`, ki vrne prevoženo razdaljo.

```python
ana = Kolesar()
ana.premik("<")
ana.premik("<")
ana.prevozi("2>")
print(ana.razdalja())
```

izpiše `4` (čeprav je Ana po vsem tem dejansko na `(0, 0)`).

#### Rešitev

Razred shranjuje samo koordinati `x` in `y`, iz česar (kot zelo jasno namiguje naloga) ne moremo izračunati, kakšno pot je kolesar prevozil. To bo očitno potrebno dodati med atribute.

Klasična skušnjava je, da atribut poimenujemo `self.razdalja`. To ne bo delovalo, ker je `razdalja` ime metode. Če v konstruktor dodamo `self.razdalja = 0`, bo `self.razdalja` zdaj očitno neko število (v začetku 0), ne pa metoda (ki jo naloga ter tudi testi pričakujejo in kličejo). Lahko pa jo poimenujemo, recimo `_razdalja`.

In [11]:
class Kolesar:
    def __init__(self):
        self.x = 0
        self.y = 0
        self._razdalja = 0

    def premik(self, smer):
        self.x += (smer == ">") - (smer == "<")
        self.y += (smer == "v") - (smer == "^")
        self._razdalja += 1

    def prevozi(self, pot):
        for koliko, kam in zip(pot[::2], pot[1::2]):
            for _ in range(int(koliko)):
                self.premik(kam)

    def lokacija(self):
        return self.x, self.y

    def razdalja(self):
        return self._razdalja

Ker `prevozi` kliče `premik`, spreminjamo (poleg konstruktorja) samo še `premik`, `prevozi` pa ostane takšen, kot je.

### 4. Turbo

Kolesarju dodajte metodi `vkljuci_turbo()` in `izkljuci_turbo()`. Ko je "turbo" vključen, so vsi premiki (prek metode `premik` in `prevozi`) podvojeni: ko ga premaknemo za eno polje, se premakne za dve. Ker turbo.

Će pokličemo `vkljuci_turbo` takrat, ko je ta že prižgan, se ne zgodi nič.

```python
ana = Kolesar()
ana.vkljuci_turbo()
ana.premik(">")  # po tem je na (2, 0)
ana.premik("v")  # po tem je na (2, 2)
ana.premik("v")  # po tem je na (2, 4)
ana.izkljuci_turbo()
ana.premik("^")  # po tem je na (2, 3)
print(ana.lokacija())
```

izpiše `(2, 3)`.

Turbo velja tudi za `prevozi`:

```python
ana = Kolesar()
ana.vkljuci_turbo()
ana.prevozi("1v3>")
print(ana.lokacija())
```

izpiše (2, 6), saj vsak premik dol in desno šteje dvojno.

#### Rešitev

To zahteva dodaten atribut, ki pove, ali je turbo vključen. Poimenovali ga bomo `turbo`. Konstruktor ga bo nastavil na `False`, novi metodi ga bosta spreminjali na `True` in `False`.

`premik` ga mora upoštevati. To bomo storili tako, da bomo premika pomnožili z nekim faktorjem, ki bo enak `1`, če je turbo izključen in `2`, če je vključen.

In [13]:
class Kolesar:
    def __init__(self):
        self.x = 0
        self.y = 0
        self._razdalja = 0
        self.turbo = False

    def premik(self, smer):
        f = 1 + self.turbo
        self.x += ((smer == ">") - (smer == "<")) * f
        self.y += ((smer == "v") - (smer == "^")) * f
        self._razdalja += f

    def prevozi(self, pot):
        for koliko, kam in zip(pot[::2], pot[1::2]):
            for _ in range(int(koliko)):
                self.premik(kam)

    def lokacija(self):
        return self.x, self.y

    def razdalja(self):
        return self._razdalja

    def vkljuci_turbo(self):
        self.turbo = True

    def izkljuci_turbo(self):
        self.turbo = False

Pri računanju faktorja (`f`), s katerim pomnožimo premike in povečujemo razdalja, spet upoštevamo, da je `True` enak `1`. V Pythonu. 

### 5. Varni turbo

Zaradi evropskih predpisov se mora turbo samodejno izključiti po petih premikih. Naredite, da bo tudi vaš razred `Kolesar` skladen z evropsko zakonodajo.

```python
ana = Kolesar()
ana.vkljuci_turbo()  # turbo bo prižgan 5 premikov
ana.premik(">")  # po tem je na (2, 0); turbo bo še 4 premike
ana.prevozi("2v")  # po tem je na (2, 4); turbo bo še 2(!) premika
ana.prevozi("3v2<")   # pot tem je na (0, 9).
                     # Turbo je veljal samo za prva dva premika dol
                     # tretji dol in in oba levo sta bila brez turba
```

Še vedno velja: če pokličemo `vkljuci_turbo` takrat, ko je ta že prižgan, se ne zgodi nič! Števec se ne resetira na 5 premikov, temveč teče naprej. Če bi v gornjem primeru pred zadnji `prevozi` dodali ponoven `vkljuci_turbo`, se ne bi nič spremenilo, turbo bi bilo še vedno vključen samo za naslednja dva premika.

#### Rešitev

Tu bi lahko uvedli dodatni atribut, ki bi povedal, koliko časa je turbo že vključen ali koliko časa naj bo še vključen. Vendar to ni potrebno: spremenimo lahko kar pomen atributa `turbo`: ta bo po novem večji od `0`, če je turbo vključen, in `0`, če ni. Vključevanje ga nastavi na `5`, izključevanje na `0`. Računanje faktorja pa spremenimo tako, `1 + self.turbo` zamenjamo z `1 + (self.turbo > 5)`. Pazite na oklepaje: izraz `1 + self.turbo > 5` bi bil enakovreden `(1 + self.turbo) > 5`, saj ima seštevanje (razumljivo!) prednost pred primerjanjem.

In [16]:
class Kolesar:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.razdalja_ = 0
        self.turbo = 0

    def premik(self, smer):
        f = 1 + (self.turbo > 0)
        self.x += ((smer == ">") - (smer == "<")) * f
        self.y += ((smer == "v") - (smer == "^")) * f
        self.turbo -= self.turbo > 0
        self.razdalja_ += f

    def prevozi(self, pot):
        for koliko, kam in zip(pot[::2], pot[1::2]):
            for _ in range(int(koliko)):
                self.premik(kam)

    def lokacija(self):
        return self.x, self.y

    def razdalja(self):
        return self.razdalja_

    def vkljuci_turbo(self):
        if not self.turbo:
            self.turbo = 5

    def izkljuci_turbo(self):
        self.turbo = 0